## Radial Basis Function (RBF) Kernel

The RBF (Radial Basis Function) kernel is widely used to measure similarity between samples.

The kernel is defined as:

$$
K(x, y) = \exp\left(-\gamma (x - y)^2\right)
$$

Where:
- $\gamma$ is the kernel bandwidth parameter
- $x, y \in \mathbb{R}$

This function decays as the squared distance between $x$ and $y$ increases,  
meaning points farther apart are considered less similar.


In [ ]:
import numpy as np

def rbf_kernel(x, y, gamma):
    return np.exp(-gamma * (x - y) ** 2)

## Kernel Matrix

Given a space of discrete values (e.g., integers from $0$ to $2^n - 1$),  
the **kernel matrix** is a symmetric matrix defined as:

$$
K_{ij} = \frac{1}{m} \sum_{k=1}^{m} \exp\left(-\gamma_k (x_i - x_j)^2\right)
$$

This matrix encodes pairwise similarity between elements in the space  
and is reused across MMD evaluations for efficiency.


In [ ]:
def compute_kernel_matrix(space: np.ndarray, gammas: np.ndarray) -> np.ndarray:
    sq_dists = np.abs(space[:, None] - space[None, :]) ** 2
    K = sum(np.exp(-gamma * sq_dists) for gamma in gammas) / len(gammas)
    return K

## Kernel Expectation Value

Given two distributions $p(x)$ and $q(x)$ represented as vectors, and a kernel matrix $K$:

$$
\mathbb{E}[K] = p^T K q
$$

This expression measures how similar the two distributions are in the feature space induced by the kernel.


In [ ]:
def kernel_expectation(px: np.ndarray, py: np.ndarray, kernel_matrix: np.ndarray) -> float:
    return px.T @ kernel_matrix @ py

## Squared Maximum Mean Discrepancy (MMD) Loss

MMD measures the distance between two probability distributions:

$$
\text{MMD}^2(p, q) = \mathbb{E}[K(x, x')] + \mathbb{E}[K(y, y')] - 2\mathbb{E}[K(x, y)]
$$

In matrix form, using kernel expectations:

$$
\text{MMD}^2 = (p - q)^T K (p - q)
$$


In [ ]:
def mmd_loss(px: np.ndarray, py: np.ndarray, kernel_matrix: np.ndarray) -> float:
    diff = px - py
    return kernel_expectation(diff, diff, kernel_matrix)

## Kullback–Leibler (KL) Divergence

KL divergence measures how much one probability distribution diverges from another:

$$
D_{KL}(p \parallel q) = \sum_{i} p_i \log\left(\frac{p_i}{q_i}\right)
$$

If $q_i = 0$, we skip that term to avoid division by zero.  
Often, we use a small $\epsilon$ to clip values and ensure numerical stability.


In [ ]:
def kl_divergence(p: np.ndarray, q: np.ndarray, eps=1e-12) -> float:
    p = np.clip(p, eps, 1.0)
    q = np.clip(q, eps, 1.0)
    return np.sum(p * np.log(p / q))


## Bitstring ↔ Integer Utilities

Used to translate between:

- Integer indices $i \in [0, 2^n)$  
- Bitstring representations $b \in \{0, 1\}^n$


In [ ]:
def int_to_bitstring(n: int, length: int) -> str:
    return format(n, f'0{length}b')

def bitstring_to_int(bitstring: str) -> int:
    return int(bitstring, 2)

## Mapping Probabilities to Bitstrings

Quantum circuits produce probability distributions over computational basis states (bitstrings).

We often want to extract the bitstring representations of non-zero (or significant) probabilities.

Given:
- A probability vector `p` of length $2^n$
- Each index $i$ corresponds to a bitstring of length $n$

We use:

$$
\texttt{bitstring}_i = \text{format}(i, '0nb')
$$

Where:

$$
n = \log_2(\text{len}(p))
$$

This helps in:
- Visualizing which basis states the circuit is predicting
- Comparing generated vs valid configurations (e.g., for Bars & Stripes)


In [ ]:
def probs_to_bitstrings(prob_vector: np.ndarray, threshold: float = 1e-6) -> list:
    n = int(np.log2(len(prob_vector)))
    return [int_to_bitstring(i, n) for i, p in enumerate(prob_vector) if p > threshold]

## Chi Validity Score

If $S$ is a set of generated samples and $V$ is a set of valid bitstrings:

$$
\chi = \frac{\text{# of valid samples in } S}{\text{total samples}}
$$

This evaluates how well a generative model captures a constrained distribution (e.g., Bars & Stripes).

This metric tells us how well a model **respects dataset constraints**, especially when only a subset of $2^n$ basis states are valid.


In [ ]:
def compute_chi(samples: list, valid_bitstrings: list) -> float:
    return np.mean([s in valid_bitstrings for s in samples])

## Probability Normalization & Stable Logarithm

### 1. Probability Normalization

Due to floating-point precision issues, a probability vector may **not sum to 1 exactly**.

To fix this, we normalize each probability value:

$$
p_i \leftarrow \frac{p_i}{\sum_j p_j}
$$

---

### 2. Safe Logarithm (Avoiding `log(0)`)

When computing metrics like **KL Divergence** or **Entropy**, we often need $\log(p_i)$.  
But:

- $\log(0) = -\infty$ → undefined  
- Causes `NaN` or `inf` in calculations

So we apply a small epsilon ($\epsilon$) to avoid zero:

$$
\log(p_i) \rightarrow \log(\max(p_i, \epsilon))
$$

A typical choice is:

$$
\epsilon = 10^{-12}
$$

This ensures **numerical stability** and avoids invalid values during computation.


In [ ]:
def normalize_probs(probs: np.ndarray) -> np.ndarray:
    total = np.sum(probs)
    return probs / total if total > 0 else probs

def safe_log(x: np.ndarray, eps=1e-12) -> np.ndarray:
    return np.log(np.clip(x, eps, None))